# Pretrained CNN Models

https://www.image-net.org/challenges/LSVRC/

**LSVRC(이미지넷 대규모 시각 인식 챌린지 Large Scale Visual Recognition Challenge)**는  
**ImageNet** 데이터셋을 이용해 이미지 분류와 객체 탐지 등 컴퓨터 비전 알고리즘의 성능을 겨루는 국제 대회이다.  
2010년부터 개최되었고, 2012년 **AlexNet**의 우승을 계기로 딥러닝이 본격적으로 주목받기 시작했다.  
이후 **VGGNet**, **GoogLeNet**, **ResNet** 등 혁신적인 딥러닝 모델들이 이 대회를 통해 등장하며 컴퓨터 비전 발전을 이끌었다.

![](https://d.pr/i/9p1so1+)


| **세대**   | **주요 모델**                              | **특징**                                                                                           |
|------------|--------------------------------------------|----------------------------------------------------------------------------------------------------|
| **1세대**  | LeNet, AlexNet, VGG                       | - Conv → ReLU → Pooling으로 이어지는 CNN 기본 구조 확립                                          |
| **2세대**  | GoogLeNet(Inception), Inception v2/v3/v4, <br>InceptionResNet, Xception | - 다양한 크기의 필터를 한꺼번에 결합<br>- 1x1 Convolution으로 계산량 감소 및 네트워크 효율성 증대 |
| **3세대**  | MobileNet(v1, v2), SeNet, NasNet, EfficientNet | - 네트워크 Depth/파라미터 감소 및 최적화<br>- Depthwise Convolution으로 계산 효율성 향상<br>- 필터 Channel 수, Kernel 크기 등의 하이퍼파라미터를 Auto Search로 최적화 |



![https://medium.com/analytics-vidhya/cnns-architectures-lenet-alexnet-vgg-googlenet-resnet-and-more-666091488df5](https://d.pr/i/HwLSVx+)


**프레임워크별 사용가능한 pre-trained models**

- keras/tensorflow: https://keras.io/api/applications/
- pytorch: https://docs.pytorch.org/vision/main/models.html


**성능지표 Top-n Accuracy, Top-n Error**

1. **Top-1 정확도**:
    - 모델이 예측한 가장 높은 확률의 클래스가 실제 정답인 경우의 비율
    - 예를 들어, Top-1 정확도가 77.1%라면, 모델이 최상위로 예측한 클래스가 77.1% 확률로 정답이라는 의미
2. **Top-5 정확도**:
    - 모델이 예측한 상위 5개의 클래스 중 하나라도 정답인 경우의 비율
    - 예를 들어, Top-5 정확도가 93.3%라면, 모델이 예측한 상위 5개의 클래스 중 하나가 정답일 확률이 93.3%라는 의미
3. **Top-5 오류율**:
    - 상위 5개의 예측 결과 중 어느 하나도 정답을 포함하지 않을 확률. **Top-5 오류율 = 1 - Top-5 정확도**로 계산한다.
    - 예를 들어, Top-5 정확도가 93.3%라면, Top-5 오류율은 6.7%이다. 이는 모델이 상위 5개 예측 결과 중 어느 하나도 정답을 포함하지 않을 확률이 6.7%라는 의미
    

In [1]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

# 1. 환경 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 10 # CIFAR10
batch_size = 64
num_epochs = 50
learning_rate = 1e-3

In [2]:
# 2. 데이터 전처리 및 DataLoader
# - 수백만 장의 ImageNet 이미지를 분석해서 구한 RGB 채널별 통계값입니다.
# - Mean (평균): Red 0.485, Green 0.456, Blue 0.406
# - Std (표준편차): Red 0.229, Green 0.224, Blue 0.225
# - torchvision.models에서 제공하는 사전 학습된 모델들(ResNet, VGG, DenseNet 등)은 모두 이 값으로 정규화된 상태에서 학습되었으므로,
# - 사용할 이미지를 이 모델에 넣을 때도 똑같은 기준으로 정규화를 해줘야 모델이 학습했던 데이터 분포와 비슷해져서 성능이 제대로 나온다.
transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])
transform_val = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
val_dataset   = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform_val)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader   = torch.utils.data.DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, num_workers=4)

/Users/jy/SKN23/multimodal/multi_venv/lib/python3.12/site-packages/torchvision/datasets/cifar.py:83: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  entry = pickle.load(f, encoding="latin1")


In [3]:
# 3. 사전학습된 모델 불러오기 및 분류기 교체
model = models.resnet18(weights=True)
print(model)

# 모든 파라미터 동결
for param in model.parameters():
    param.requires_grad = False

# FC 레이어 교체
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes) # required_grad=True 학습대상
model = model.to(device)

# 학습 대상 파라미터 확인
params_to_update = [p for p in model.parameters() if p.requires_grad]
print('학습할 파라미터: ')
params_to_update

/Users/jy/SKN23/multimodal/multi_venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

[Parameter containing:
 tensor([[ 0.0182, -0.0179, -0.0409,  ..., -0.0142, -0.0084,  0.0287],
         [-0.0125, -0.0411, -0.0369,  ..., -0.0194, -0.0274,  0.0155],
         [-0.0230,  0.0300,  0.0428,  ...,  0.0063, -0.0386, -0.0069],
         ...,
         [-0.0038, -0.0392, -0.0025,  ...,  0.0382, -0.0238,  0.0254],
         [ 0.0013, -0.0054,  0.0158,  ...,  0.0267,  0.0205, -0.0061],
         [ 0.0277, -0.0425, -0.0413,  ..., -0.0440, -0.0157,  0.0253]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0115, -0.0268, -0.0408,  0.0381, -0.0372,  0.0433,  0.0039,  0.0116,
         -0.0321, -0.0341], requires_grad=True)]

In [ ]:
import numpy as np
import torch

class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

# 4. 손실함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params_to_update, lr=learning_rate)

# 스케줄러: 검증 손실이 정체될 때 학습률을 0.1배로 감소시킴
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

# Early Stopping 객체 생성
early_stopping = EarlyStopping(patience=5, verbose=True, path='best_model.pt')

# 5. 학습 및 검증 함수 정의
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def validate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# 6. 메인 학습 루프
for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_acc     = validate(model, val_loader, criterion)

    print(f"Epoch {epoch}/{num_epochs} | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # Learning Rate Scheduler 단계 업데이트
    scheduler.step(val_loss)

    # Early Stopping 단계 업데이트
    early_stopping(val_loss, model)

    if early_stopping.early_stop:
        print("Early stopping triggered. Training stopped.")
        break

# 7. 최적의 모델 불러오기 및 최종 테스트
model.load_state_dict(torch.load('best_model.pt'))
test_loss, test_acc = validate(model, val_loader, criterion)
print(f"Final Test Acc (Best Model): {test_acc:.4f}")

 29%|██▉       | 229/782 [09:15<19:45,  2.14s/it]